In [2]:
import os
import re
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from sklearn.utils import shuffle

def getDataFromFiles(filefolder, category1 = "*", category2= "*", error_del = False, min_len=100, drop_duplicates=False, maxrow_by_category1 = 0, maxrow_by_category2 = 0) :
    if category1 == "*" : 
        category1 = set([re.search('_.*_', str(filename)).group().replace("_","") for filename in Path(filefolder).rglob("*.json")])
    else : 
        category1 = category1.split(",")
        
    if category2 == "*" : 
        category2 = set([re.search('_.*\.json', str(filename)).group().replace(".json","").split("_")[2] for filename in Path(filefolder).rglob("*.json")])
    else : 
        category2 = category2.split(",")
    
    df_li = []
    for cate1 in category1 :
        for cate2 in category2 :
            pattern = "*_{}_{}.json".format(cate1, cate2)
            
            tmp_df_li = []
            for path in Path(filefolder).rglob(pattern):
                try : 
                    tmp_df = pd.read_json(open(path, "r", encoding="utf8"))
                    tmp_df = tmp_df[tmp_df['content'].apply(len) > min_len] #지정 길이 이상의 데이터만 조회
                    tmp_df = tmp_df[tmp_df['category1'].apply(len) < 20]
                    tmp_df = tmp_df[tmp_df['category2'].apply(len) < 20]
                    if drop_duplicates : #중복 뉴스 삭제
                        tmp_df = tmp_df.drop_duplicates(subset='content', keep='first')
                    tmp_df_li.append(tmp_df)
                except Exception as e:
                    if error_del : os.remove(path) #오류가 있는 파일 삭제. 재작업 목적
                    print("Error file : {}".format(path))
                    print("Error message : {}".format(str(e)))
            if len(tmp_df_li) > 0 :                
                # 세분류별 데이터 건수 제한
                tmp_df = pd.concat(tmp_df_li, sort=False)
                print("-- Parttern : {} [{}] --".format(pattern, tmp_df.shape[0]))
                
                if (tmp_df.shape[0] > 0) :
                    if maxrow_by_category2 > 0 : tmp_df = shuffle(tmp_df)[:maxrow_by_category2]
                    df_li.append(tmp_df)    
                    
    df = pd.concat(df_li, sort=False)
    print("{} rows returned".format(df.shape[0]))
    return df

df = pd.DataFrame()
df = getDataFromFiles("../../NaverNews/navernews/news/news/2019", category1='*', category2='*'
                      , min_len=100, drop_duplicates=True, error_del = False, maxrow_by_category2 = 1000)
df.to_csv('./data/2019news.csv', index=False)

-- Parttern : *_IT과학_보안해킹.json [7334] --
-- Parttern : *_IT과학_모바일.json [15708] --
-- Parttern : *_IT과학_인터넷SNS.json [23126] --
-- Parttern : *_IT과학_게임리뷰.json [27513] --
-- Parttern : *_IT과학_통신뉴미디어.json [29069] --
-- Parttern : *_IT과학_IT일반.json [117554] --
-- Parttern : *_IT과학_과학일반.json [22402] --
-- Parttern : *_IT과학_컴퓨터.json [10877] --
-- Parttern : *_정치_북한.json [61606] --
-- Parttern : *_정치_청와대.json [75001] --
-- Parttern : *_정치_국회정당.json [209419] --
-- Parttern : *_정치_정치일반.json [394251] --
-- Parttern : *_정치_행정.json [58129] --
-- Parttern : *_정치_국방외교.json [89047] --
-- Parttern : *_사회_사건사고.json [141907] --
-- Parttern : *_사회_인권복지.json [18339] --
-- Parttern : *_사회_지역.json [890560] --
-- Parttern : *_사회_노동.json [33076] --
-- Parttern : *_사회_인물.json [65802] --
-- Parttern : *_사회_사회일반.json [905666] --
-- Parttern : *_사회_식품의료.json [37425] --
-- Parttern : *_사회_언론.json [8037] --
-- Parttern : *_사회_환경.json [31317] --
-- Parttern : *_사회_교육.json [64496] --
-- Parttern : *_생활문화_종교.json [14474

In [64]:
n = 1000

tmp = df[['url','category1','category2','date','title','media','content']]

li = []
li.append(shuffle(tmp[tmp.category1=="IT/과학"])[:n])
li.append(shuffle(tmp[tmp.category1=="정치"])[:n])
li.append(shuffle(tmp[tmp.category1=="사회"])[:n])
li.append(shuffle(tmp[tmp.category1=="경제"])[:n])
li.append(shuffle(tmp[tmp.category1=="생활/문화"])[:n])
tmp_df = pd.concat(li, sort=False)

tmp_df.to_csv('./data/2019news_{}.csv'.format(n), index = False)
tmp_df.groupby([tmp_df['category1']])['url'].agg(['count'])

,count
category1,
IT/과학,1000
경제,1000
사회,1000
생활/문화,1000
정치,1000


In [70]:
df = pd.read_csv('./data/2019news_5000.csv')
df.groupby([df['category1']])['url'].agg(['count'])
#df.groupby([df['category1'], df['category2']])['url'].agg(['count'])

,count
category1,
IT/과학,5000
경제,5000
사회,5000
생활/문화,5000
정치,5000


In [71]:
df.head()

,url,category1,category2,date,title,media,content
0,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,게임/리뷰,2019-10-10,"[이슈] 헝그리앱, 인기 MMORPG '비천무협' 탈것 관련 퀴즈 이벤트 진행",데일리e스포츠,"본 기사는 최근 대세로 떠오른 신규, 인기 스마트폰 게임과 관련 방송 콘텐츠 등을 ..."
1,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,보안/해킹,2019-11-21,"과기정통부-KISIA, 2019 정보보호산업인의 밤 개최",전자신문,2019 정보보호산업인의 밤 포스터. 과학기술정보통신부 제공 과학기술정보통신부(장관...
2,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,통신/뉴미디어,2019-03-22,[인사]과학기술정보통신부,이데일리,[이데일리 김현아 기자]◇ 고공단 승진 △과학기술정보통신부(파견) 허원석(전파정책기...
3,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,게임/리뷰,2019-01-31,모바일게임의 자동전투는 필요악(惡)일까?,동아일보,이제는 국내 게임시장의 중심으로 떠오른 모바일게임의 해묵은 논란이 있다. 이제는 모...
4,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,게임/리뷰,2019-01-08,"'삼국지 천하무적', 장수 제갈량과 허저 추가",포모스,위드허그는 모바일 액션RPG '삼국지 천하무적'에 장수 제갈량과 허저를 추가하는 업...


In [54]:
import re

cnt = 0
for i in range(0, df.shape[0]) :
    cont = df['content'][i]
    #print("-"*50)
    #print(cont)
    #print("-")
    #사진=최혁 기자
    ad_re = re.compile(r'(사진=.{0,20}기자[ \,])')
    cont = ad_re.sub('', cont)
    
    
    src = re.search(r'(\..{0,30}기자[ \,])|( \<.{0,30}기자\>)' ,cont)
    if src : cont = cont[:src.span()[0]+1]
    if cont.find(r'{}') > 0 : cont = cont[cont.find(r'{}')+2:] #특수문자
    #cont = cont[:cont.find(r'▶')] #특수문자
    #cont = cont[:cont.find(r'☞')] #특수문자
    #cont = cont[:cont.find(r'ⓒ')] #특수문자
    #cont = cont[:cont.find(r'<ⓒ')] #특수문자
    cont = cont.strip()
            
    #print(cont)
    #print("-"*50)
    #cnt = cnt + 1
    df['content'][i] = cont
    #if cnt == 20 :
    #    break

/Users/leeminho/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [60]:
df[['url','category1','category2','date','title','media','content']].head()


,url,category1,category2,date,title,media,content
0,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,보안/해킹,2019-12-08,도이치뱅크 10년내 법정화폐 붕괴…BIS 디지털화폐 받아들여야,한국경제,아구스틴 카르스텐스 국제결제은행(BIS) 사무총장(사진=연합뉴스) 기존 법정화폐의 ...
1,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,보안/해킹,2019-07-10,내년 국제 올림픽때 韓 콜드월렛 쓰이나 …시장 노크,아이뉴스24,"도쿄 올림픽에 적용, 펜타시큐리티 등 수출 '타진' [아이뉴스24 최은정 기자] 일..."
2,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,보안/해킹,2019-04-30,"[코인터뷰] 보상받는 SNS 포레스팅, 페북·아프리카TV 기능 한곳에",한국경제,엄재현 포레스팅HQ 대표 30일 오픈베타 서비스 출시 [ 김봉구 기자 ] 지난 2...
3,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,보안/해킹,2019-04-15,IMF·WB가 가상화폐 발행? 사내교육용,한국경제,[ 김산하 기자 ] 국제통화기금(IMF)과 세계은행(WB)이 가상화폐(암호화폐)를...
4,https://news.naver.com/main/read.nhn?mode=LS2D...,IT/과학,보안/해킹,2019-11-20,"센스톤, 인도네시아 PG사에 차세대 인증기술 공급",전자신문,"(왼쪽부터) 유창훈 센스톤 대표, 나빌라 알사고프 도쿠 설립자, 히멜다 르누앗 도쿠..."


In [8]:
df.to_csv('./data/2019.csv', index=False)

In [2]:
a = df[df['category1'] not in ['IT/과학','경제','사회','생활/문화','정치']]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
df.shape

In [3]:
df.groupby([df['category1']])['url'].agg(['count'])

,count
category1,
IT/과학,8000
경제,8000
사회,10000
생활/문화,11000
정치,6000


In [61]:
df.groupby([df['category1'], df['category2']])['url'].agg(['count'])

count
category1 category2       
IT/과학     IT 일반       1000
          게임/리뷰       1000
          과학 일반       1000
          모바일         1000
          보안/해킹       1000
          인터넷/SNS     1000
          컴퓨터         1000
          통신/뉴미디어     1000
경제        경제 일반       1000
          글로벌 경제      1000
          금융          1000
          부동산         1000
          산업/재계       1000
          생활경제        1000
          중기/벤처       1000
          증권          1000
사회        교육          1000
          노동          1000
          사건사고        1000
          사회 일반       1000
          식품/의료       1000
          언론          1000
          인권/복지       1000
          인물          1000
          지역          1000
          환경          1000
생활/문화     건강정보        1000
          공연/전시       1000
          날씨          1000
          도로/교통       1000
          생활문화 일반     1000
          여행/레저       1000
          음식/맛집       1000
          자동차/시승기     1000
          종교          1000
          책           1000
          패션/뷰티       1000
정치        국방/외교       1000
          국회/정당       1000
          북한          1000
          정치일반        1000
          청와대         1000
          행정          1000